# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_path="../output_data/weather_data.csv"
weather_data_df=pd.read_csv(weather_data_path,encoding="utf-8")
weather_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        560 non-null    object 
 1   lng         560 non-null    float64
 2   lat         560 non-null    float64
 3   Max Temp    560 non-null    float64
 4   Humidity    560 non-null    int64  
 5   Wind Speed  560 non-null    float64
 6   Cloudiness  560 non-null    int64  
 7   Country     555 non-null    object 
 8   Date        560 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 39.5+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_data_df[["lat", "lng"]]

# Fill NaN values and convert to float
humidity = weather_data_df["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                max_intensity=100,
                                 dissipating=True,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# A max temperature between 15 to 30.
# Wind speed less than 15 mph.
# Humidity in between 25-50
# cloudiness in beween 10-35
Ideal_weather=weather_data_df.loc[(weather_data_df["Max Temp"]>15)&(weather_data_df["Max Temp"]<30)
                                  &(weather_data_df["Wind Speed"]<15)
                                  &(weather_data_df["Humidity"]>20)&(weather_data_df["Humidity"]<50)
                                  &(weather_data_df["Cloudiness"]>10)&(weather_data_df["Cloudiness"]<35)]
Ideal_weather.dropna(inplace = False) 
Ideal_weather.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 17 to 536
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        10 non-null     object 
 1   lng         10 non-null     float64
 2   lat         10 non-null     float64
 3   Max Temp    10 non-null     float64
 4   Humidity    10 non-null     int64  
 5   Wind Speed  10 non-null     float64
 6   Cloudiness  10 non-null     int64  
 7   Country     9 non-null      object 
 8   Date        10 non-null     int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 800.0+ bytes


In [9]:
Ideal_weather.head()

,city,lng,lat,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
17,Mantamádos,26.3339,39.3117,26.07,46,1.69,11,GR,1623487183
29,Blagaj,17.8886,43.2575,23.98,43,2.57,20,BA,1623487185
45,Şebinkarahisar,38.4236,40.2883,20.15,45,2.19,15,TR,1623487189
123,Mafeteng,27.2374,-29.8230,15.09,40,3.09,12,LS,1623487210
124,Krasnovishersk,57.0820,60.4078,25.00,26,2.94,32,RU,1623487210


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df=Ideal_weather[["city","lng","lat"]]
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 17 to 536
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   city    10 non-null     object 
 1   lng     10 non-null     float64
 2   lat     10 non-null     float64
dtypes: float64(2), object(1)
memory usage: 640.0+ bytes


In [11]:
# set up additional columns to hold information
hotel_df['Hotel Name'] = ""
hotel_df['Address'] = ""


hotel_df.head(1)

C:\Users\mailr\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mailr\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,city,lng,lat,Hotel Name,Address
17,Mantamádos,26.3339,39.3117,,


In [14]:
hotel_df[["lng","lat"]]

,lng,lat
17,26.3339,39.3117
29,17.8886,43.2575
45,38.4236,40.2883
123,27.2374,-29.8230
124,57.0820,60.4078
186,68.3333,51.8000
306,16.3410,-9.5402
524,43.2500,62.8000
534,87.6005,43.8010
536,19.1167,-24.1333


In [38]:
# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Find a hotel based on city_coords.
target_search = "Hotels"
radius = 5000
city_coords = hotel_df[["lng","lat"]]

# redefine params
params = {
    "location": city_coords,
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat=row["lat"]
    lng=row["lng"]
    city=row["city"]
    # get city location from df
    city_coords = f"{lat}, {lng}"
    print (city_coords)
    # add keyword to params dict
    params['location'] = city_coords

    # assemble url and make API request
   
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'address'] = results[0]['vicinity']
        print(f"Closest hotel is {results[0]['name']} for {city} .")
        print(f"Closest hotel {results[0]['name']} address is {results[0]['vicinity']} .")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


39.3117, 26.3339


C:\Users\mailr\anaconda3\envs\pythondata\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\mailr\anaconda3\envs\pythondata\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is PANORAMA RESORT - ΞΕΝΟΔΟΧΕΙΟ - Ε.ΠΑΛΑΙΟΛΟΓΟΥ ΚΑΙ ΣΙΑ ΤΟΥΡΙΣΤΙΚΑ ΚΑΤΑΛΥΜΑΤΑ Ο.Ε. - ΛΕΣΒΟΣ for Mantamádos .
Closest hotel PANORAMA RESORT - ΞΕΝΟΔΟΧΕΙΟ - Ε.ΠΑΛΑΙΟΛΟΓΟΥ ΚΑΙ ΣΙΑ ΤΟΥΡΙΣΤΙΚΑ ΚΑΤΑΛΥΜΑΤΑ Ο.Ε. - ΛΕΣΒΟΣ address is Ασπροπόταμος, ΛΕΣΒΟΣ .
43.2575, 17.8886
Closest hotel is Hotel Blagaj for Blagaj .
Closest hotel Hotel Blagaj address is Branilaca Bosne, Blagaj .
40.2883, 38.4236
Closest hotel is Apart Hotel Tamzara for Şebinkarahisar .
Closest hotel Apart Hotel Tamzara address is Tamzara Mah Prof. Dr. Özsan Cad. No:60 /A, Şebinkarahisar .
-29.823, 27.2374
Missing field/result... skipping.
60.4078, 57.082
Missing field/result... skipping.
51.8, 68.3333
Closest hotel is Atbasar for Atbasar .
Closest hotel Atbasar address is Chokan Valikhanov st. 11, Atbasar .
-9.5402, 16.340999999999998
Closest hotel is Palanca Hotel for Malanje .
Closest hotel Palanca Hotel address is Malanje .
62.8, 43.25
Missing field/result... skipping.
43.801, 87.6005
Closest hotel is JinJiang Int

In [34]:
# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Find a hotel based on city_coords.
target_search = "Hotels"
radius = 5000
# city_coords =[-24.1333,19.1167]

# redefine params
params = {
    "location": city_coords,
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}

# use iterrows to iterate through pandas dataframe

city_coords =f"-24.1333,19.1167"
print (city_coords)
    # add keyword to params dict
params['location'] = city_coords

    # assemble url and make API request
   
response = requests.get(base_url, params=params).json()
response

-24.1333,19.1167


{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -24.1424851, 'lng': 19.1176057},
    'viewport': {'northeast': {'lat': -24.14112877010728,
      'lng': 19.11895667989273},
     'southwest': {'lat': -24.14382842989272, 'lng': 19.11625702010728}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Ley-Lia Guest House',
   'photos': [{'height': 3024,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113337269990893982226">Naftal Hamata</a>'],
     'photo_reference': 'ATtYBwKoqWzWQaw7krYsZM0jpzTSC2MtQxG5M7FwtvsM-uCp6Ucr6YyyDz-izYxYI1ejtpY53DJCgjH6UEg-22nt2Sjd0KiNbHFkWj5z0_QeFGwXDrJiM-wirJKekIehLrgfuMGjGbWqRc58WtoxK2FvTYxlCohcjA0zIR2HROGFERf6J8FV',
     'width': 4032}],
   'place_id': 'ChIJBWihHWDBDxwR9hz3YVLB3LQ',
   'plus_code': {'compound_code': 'V459+22 Aranos, Namibia',
    'global_code': '5F7XV459+22'},
   'rating': 4.6,
   'reference': 'ChIJBWihHWDB

In [37]:
response.keys()

dict_keys(['html_attributions', 'results', 'status'])

In [33]:
city_coords =f"-24.1333,19.1167"
city_coords

'-24.1333,19.1167'

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
